In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [25]:
import input_data
mnist = input_data.read_data_sets('data/',one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
[2051] 60000 28 28


Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
[2051] 10000 28 28
Extracting data/t10k-labels-idx1-ubyte.gz


In [10]:
# 定义输入输出格式
n_input = 784
n_output = 10
weights = {
    'wc1' : tf.Variable(tf.random_normal([3,3,1,64],stddev=0.1)),  #(H,W,in_channel,out_channel) D:deepth N:输出的label数，这一个卷积层输出为为64  3，3对应filter
    'wc2' : tf.Variable(tf.random_normal([3,3,64,128],stddev=0.1)), # 上一层的输出为64层
    'wd1' : tf.Variable(tf.random_normal([7*7*128,1024],stddev=0.1)), # 28->卷积1->->pooling1->j卷积2->->pooling2  H变成7
    'wd2' : tf.Variable(tf.random_normal([1024,n_output],stddev=0.1))
 }

biases = {
    
    'bc1' : tf.Variable(tf.random_normal([64],stddev=0.1)),
    'bc2' : tf.Variable(tf.random_normal([128],stddev=0.1)),
    'bd1' : tf.Variable(tf.random_normal([1024],stddev=0.1)),
    'bd2' : tf.Variable(tf.random_normal([n_output],stddev=0.1))
}

In [11]:
import pandas as pd


In [12]:
# 正向传播定义
def conv_basic(input, _w, _b, _keepratio):
    # input reshape 为tf能接受的维度
    _input_r = tf.reshape(input, shape=[-1, 28, 28, 1])  #(N,H,W,C)  N=-1定为动态，其他定好，第一维是可以算出来  C:channel  灰度图，只有1

    _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1, _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    
    # print('die')
    _conv2 = tf.nn.conv2d(_pool_dr1, _w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2, _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)

    _dense1 = tf.reshape(_pool_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])

    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)

    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])

    # return 
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1, 'pool_dr1': _pool_dr1,
        'conv_2': _conv2, 'pool2': _pool2, 'pool_dr2': _pool_dr2, 'dense1': _dense1,
        'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out
    }
    return out
print('finish function def')


finish function def


In [26]:
x = tf.placeholder(tf.float32, [None, n_input])  #n_input个输入
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

_pred = conv_basic(x, weights, biases, keepratio)['out']
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

_corr = tf.equal(tf.argmax(_pred, 1), tf.argmax(y, 1))
acc = tf.reduce_mean(tf.cast(_corr, tf.float32))

init = tf.global_variables_initializer()

print('graph head')


graph head


In [ ]:
# init = tf.global_variables_initializer()
with tf.Session() as sess:
    training_epochs = 15
    batch_size = 16
    display_step = 10
    sess.run(init)

    for epoch in range(training_epochs):
        avg_loss = 0
        one_round_batch = int(mnist.train.num_examples / batch_size)
        print(one_round_batch)
        one_round_batch = 10

        for i in range(one_round_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys,keepratio:0.7})
            avg_loss += sess.run(loss, feed_dict={x: batch_xs, y: batch_ys, keepratio: 1}) 
        avg_loss = avg_loss / one_round_batch

        if epoch % display_step == 0:
            print('epoch:%03d/%03d  loss: %.9f' % (epoch, training_epochs, avg_loss))
            train_acc = sess.run(acc, feed_dict={x: batch_xs, y: batch_ys, keepratio: 1})
            print('accuracy: %.3f' % (train_acc))

print('finished!')
            

3437


KeyboardInterrupt: 